<img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/horizontal-primary-light.png" alt="he-black-box" width="600"/>


# Homomorphic Encryption using Duet: Data Owner
## Tutorial 2: Encrypted image evaluation



Welcome!
Welcome! This tutorial will show you how to evaluate Encrypted images using Duet and TenSEAL. This notebook shows the Data Scientist view on the operations.


We recommend going through Tutorial 0 and 1 before trying this one.

## Setup

All modules are imported here, make sure everything is installed by running the cell below.

In [1]:
import pickle
import requests
import os

import syft as sy
import tenseal as ts
from syft.grid.client.client import connect
from syft.grid.client.grid_connection import GridHTTPConnection
from syft.core.node.domain.client import DomainClient
from typing import Dict

sy.load("tenseal")


## Connect to PyGrid

Connect to PyGrid Domain server.

In [2]:
client = connect(
    url="http://localhost:5000", # Domain Address
    credentials={"email":"user@email.com", "password":"userpwd"},
    conn_type= GridHTTPConnection, # HTTP Connection Protocol
    client_type=DomainClient) # Domain Client type

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 1 : Now STOP and run the Data Owner notebook until the next checkpoint.

### Define The Model

Here we define the model used by the Data Scientist. he model contains a convolution and two fully connected layers.

The input should be an encoded 28x28 matrix representing the image.
TenSEAL can be used for encoding `tenseal.im2col_encoding(ctx, input_matrix, 7, 7, 3)`
The input should also be normalized with a mean=0.1307 and an std=0.3081 before encryption.

In [3]:
# Load a pretrained model and adapt the forward call for encrypted input
class ConvMNIST():
    def __init__(self, parameters: Dict[str, list]):
        self.conv1_weight = parameters["conv1_weight"]
        self.conv1_bias = parameters["conv1_bias"]
        self.fc1_weight = parameters["fc1_weight"]
        self.fc1_bias = parameters["fc1_bias"]
        self.fc2_weight = parameters["fc2_weight"]
        self.fc2_bias = parameters["fc2_bias"]
        self.windows_nb = parameters["windows_nb"]

    def forward(self, enc_x: ts.CKKSVector) -> ts.CKKSVector:
        # conv layer
        channels = []
        for kernel, bias in zip(self.conv1_weight, self.conv1_bias):
            y = enc_x.conv2d_im2col(kernel, self.windows_nb) + bias
            channels.append(y)
        out = ts.CKKSVector.pack_vectors(channels)
        # squaring
        out.square_()
        # no need to flat
        # fc1 layer
        out = out.mm_(self.fc1_weight) + self.fc1_bias
        # squaring
        out.square_()
        # output layer
        out = out.mm_(self.fc2_weight) + self.fc2_bias
        return out

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

### Load the pretrained parameters

In [4]:
model_path = "parameters/ConvMNIST-0.1.pickle"

def download_parameters():
    try:
        os.mkdir("parameters")
    except BaseException as e:
        pass

    url = "https://github.com/OpenMined/TenSEAL/raw/master/tutorials/parameters/ConvMNIST-0.1.pickle"
    
    r = requests.get(url)

    with open(model_path, 'wb') as f:
        f.write(r.content)
        
def load_parameters() -> dict:
    try:
        download_parameters()
        parameters = pickle.load(open(model_path, "rb"))
        print(f"Model loaded from '{model_path}'")
    except OSError as ose:
        print("error", ose)
        raise ose
    return parameters


parameters = load_parameters()
model = ConvMNIST(parameters)

Model loaded from 'parameters/ConvMNIST-0.1.pickle'


### Request the image for evaluation

In [5]:
client.store.pandas

,ID,Tags,Description,object_type
0,<UID: 8ed84653b176476eae9e2a8586d08f8e>,[context],,<class 'tenseal.enc_context.Context'>
1,<UID: 620901d47e4241218b756a9daf47e33d>,[enc_image],,<class 'tenseal.tensors.ckksvector.CKKSVector'>


In [6]:
ctx_ptr = client.store["context"]
enc_image_ptr = client.store["enc_image"]

ctx_ptr.request(reason="I would like to get the context")
enc_image_ptr.request(reason="I would like to get encrypted evaluation image")

In [7]:
client.requests.pandas

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,[context],I would like to get the context,<UID: 842e3e7bf1d042ccaad6e77800a101fb>,<UID: 8ed84653b176476eae9e2a8586d08f8e>,<class 'tenseal.enc_context.Context'>
1,[enc_image],I would like to get encrypted evaluation image,<UID: 9e75e93e11d64414a32b7f9431efbeda>,<UID: 620901d47e4241218b756a9daf47e33d>,<class 'tenseal.tensors.ckksvector.CKKSVector'>


### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 2 : Now STOP and run the Data Owner notebook until the next checkpoint.

### Evaluate the encrypted image

In [8]:
enc_image = enc_image_ptr.get(delete_obj=False)
ctx = ctx_ptr.get(delete_obj=False)

enc_image.link_context(ctx)

encrypted_result = model(enc_image)

### Send the encrypted result back to the DO instance

In [9]:
encrypted_result_ptr = encrypted_result.send(client, pointable=True, tags=["result"])


[<syft.proxy.tenseal.ContextPointer object at 0x7f6734783ee0>, <syft.proxy.tenseal.CKKSVectorPointer object at 0x7f6734783550>, <syft.proxy.tenseal.CKKSVectorPointer object at 0x7f6734783d90>]

In [10]:
client.store.pandas

,ID,Tags,Description,object_type
0,<UID: 8ed84653b176476eae9e2a8586d08f8e>,[context],,<class 'tenseal.enc_context.Context'>
1,<UID: 620901d47e4241218b756a9daf47e33d>,[enc_image],,<class 'tenseal.tensors.ckksvector.CKKSVector'>
2,<UID: 4654735a9bb24354a293c53ba05438ab>,[result],,<class 'tenseal.tensors.ckksvector.CKKSVector'>


### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 3 : Now STOP and run the Data Owner notebook until the next checkpoint.

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft and TenSEAL on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)
- [Star TenSEAL](https://github.com/OpenMined/TenSEAL)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org). #lib_tenseal and #code_tenseal are the main channels for the TenSEAL project.

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)